# Perceptual Losses for Real-Time Style Transfer and Super-Resolution
(실시간 style transfer와 초고해상도를 위한 지각 손실 함수)

# Abstract
- 우리는 입력 이미지가 출력 이미지로 변환되는 이미지 변환 문제(image transformation problem)를 고려한다.
- 이러한 문제에 대한 최신의 방법은 일반적으로 feed-forward convolutional neural network를 훈련시킨다. 출력 이미지와 지상 실측(ground-truth) 이미지 사이의 픽셀당 손실(per-pixel loss)를 사용해서.
- 병렬작업(parallel work)은 고품질의 이미지를 생성가능하다. 사전에 훈련된(pretrained) 네트워크로부터 추출된 고수준 feature를 기반으로 지각 손실함수(perceptual loss function)을 정의하고 최적화함으로써.
- 우리는 두 접근방식(approach)의 장점을 결합한다. 그리고, 이미지 변환 작업을 위한 feed-forward network를 훈련하기 위한 지각 손실함수(perceptual loss function)을 제안한다.
- 우리는 image style transfer의 결과를 보여준다. 실시간으로 Gatys 등등이 제안한 최적화 방법을 해결하기 위해 feed-forward network는 훈련된다.
- 최적화 기반 방법(optimization-based method)과 비교하여, 우리의 네트워크는 비슷한 질적인(qualitative) 결과를 준다. 하지만, 3배 규모(magnitude)로 더 빠르다.
- 우리는 또한 단일 이미지(single-image) 초고해상도(super-resolution)을 실험한다. 픽셀당 손실(per-pixel loss)을 지각 손실(perceptual loss)로 대체하는 것은 시각적으로 만족스러운(pleasing) 결과를 준다.

# 1. Introduction
- 많은 고전적인(classic) 문제는 이미지 변한 작업으로써 구성될(frame) 수 있다. 시스템은 몇 개의 입력 이미지를 받고 출력 이미지로 변환한다.
- 이미지 처리(image processing)
  - 예) 노이즈 제거(denoising), 초고해상도(super-resolution), 채색(colorization)
  - 입력 : 저하된 이미지(degraded image)
    - 예) 노이즈(noisy), 저해상도(resolution), 회색조(grayscale)
  - 출력 : 고품질 컬러 이미지
- 컴퓨터 비전(computer vision)
  - 예) 의미론적 분할(semantic segmentation), 깊이 추정(depth estimation)
  - 입력 : 컬러 이미지
  - 출력 : 장면(scene)에 대한 의미론적인 또는 기하학적인 정보를 인코딩한다.
- 이미지 변환 작업(image transformation task)을 해결하기 위한 1가지 접근방식은 지도학습 방법(supervised manner)로 feedforward convolutional neural network를 훈련시키는 것이다. 픽셀당 손실 함수를 사용하여 출력 이미지와 실측 이미지 간의 차이를 측정한다.
- 이러한 접근 방식은 초해상도(super-resolution)을 위해 Dong 등등에 의해 사용되었다. 채색(colorization)을 위해 Cheng 등등에 의해 사용되었다. 분할(segmentation)을 위해 Long 등등에 의해 사용되었다. 깊이(depth) 및 표면 정규 예측(법선 벡터 예측, surface normal prediction)을 위해 Eigen 등등에 의해 사용되었다.
- 이러한 접근 방식은 테스트 시간(test-time)에 효율적이다. 오직 훈련된(trained) 네트워크를 통한 전달(forward pass)만 필요하다.
- 하지만, 이러한 방법에서 사용되는 픽셀당 손실은 출력 이미지와 실측 이미지 사이의 지각적 차이(perceptual diference)를 포착하지 못한다.
- 예를 들어, 1개의 픽셀만큼 서로 상쇄된(offset) 2개의 동일한 이미지를 고려해보자. ; 드릉릐 지각적인 유사성에도 불구하고, 그들은 아마도 픽셀당 손실에 의해 측정될 때 매우 다를 것이다.
- 이와 동시에, 최근 연구에 따르면, 고품질의 이미지는 픽셀 간의 차이 대신에 지각 손실 함수를 사용하여 만들 수 있다고 한다. 그러나 대신에, 사전학습된 컨볼루젼 신경망에서 추출된 고수준 이미지 특징(feature) 표현(representation) 간의 차이에 대한 것이다.
- 이미지는 손실 함수를 최소화함으로써 생성된다.
- 이 전략(strategy)는 Mahendran 등등에 의한 특징 전도(feature inversion), Simonya, Yosinski 등등에 의한 특징 시각화(feature visualization), Gatys 등등에 의한 텍스쳐 합성(texture synthesis)과 스타일 전달(style transfer)에 적용되었다.
- 이러한 접근 방식은 고품질 이미지를 생성한다. 하지만, 느리다. 왜냐하면, 추론(inference)은 최적화 문제를 해결해야 하기 때문이다.
- 이 논문에서, 우리는 이러한 2가지 접근 방식의 장점을 결합했다.
- 우리는 이미지 변환 작업을 위해 feedforward 변환 네트워크를 훈련시켰다. 하지만, 낮은 수준의(low-level) 픽셀 정보에만 의존하는 픽셀당 손실 함수를 사용하는 대신, 우리는 사전훈련된 손실 네트워크(pretrained loss network)의 고수준 특징에 의존하는 지각 손실 함수(perceptual loss function)을 사용하여 네트워크를 훈련한다.
- 훈련 중에, 지각 손실은 픽셀당 손실보다 이미지 유사성(image similarity)을 더 강력하게(robustly) 측정한다. 그리고, 테스트 시간(test-time)에, 변환 네트워크가 실시간으로(real-time) 작동된다.
- 우리는 2가지 작업을 실험한다.: 스타일 전송(style transfer), 단일 이미지 초고해상도
- 두 작업 다 본질적으로(inherently) 타당하지 않다.(ill-posed); 스타일 전송의 경우 올바른 단일 출력이 없다. 그리고 초고해상도의 경우 동일한 저해상도 입력을 생성할 수 있는 고해상도 이미지가 있다.
- 두 작업에서 성공하려면 입력 이미지에 대한 의미론적 추론(semantic reasoning)이 필요하다.
- 스타일 전송의 경우, 출력은 입력과 의미상(semantically) 유사해야 한다. 색상, 질감의 급격한(drastic) 변화에도 불구하고.
- 초고해상도의 경우, 시각적으로 애매모호한 저해상도 입력에서 질 높은(fine) 세부사항(detail)을 추론(infer)해야한다.
- 원칙적으로, 각각의 작업에 훈련된 고용량(high-capacity) 신경망은 관련 의미론(relevant semantic)에 대해 추론(reason)하는 방법을 암시적으로(implicitly) 학습할 수 있다. 하지만, 실제는, 우리는 처음(스크래치, scratch)부터 학습할 필요가 없다.: 지각 손실 함수의 사용은 손실 네트워크(loss network)에서 변환 네트워크(transformation network)로 의미 지식(semantic knowledge)을 전달할 수 있다.
- 스타일 전송을 위해, 우리의 feed-forward network는 논문 "Gatys, L.A., Ecker, A.S., Bethge, M.: A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576 (2015)
"의 최적화 문제를 해결하도록 훈련되었다. ; 우리의 결과는 질적으로 그리고 목적 함수(=손실 함수, objective function) 값에 의해 측정된 것과 유사하다. 하지만, 생성하는데 걸리는 속도는 3배 빠르다.
- 초고해상도의 경우, 우리는 픽셀당 손실을 지각 손실로 대체하는 것이 x4, x8 초고해상도에서 시각적으로 만족스러운 결과를 준다는 것을 보여준다.
- <img src="https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig1_HTML.gif">
  
  - 그림 1.
  - 위 그림 : 스타일 전송(style transfer) 예시
  - 아래 그림 : x4 초고해상도(super-resolution) 예시
  - 스타일 전송(style transfer)의 경우, Gatys 등등의 논문 "Gatys, L.A., Ecker, A.S., Bethge, M.: A neural algorithm of artistic style. arXiv preprint arXiv:1508.06576 (2015)
"과 유사한 결과를 얻지만 3배 더 빠르다.
  - 초해상도의 경우, 지각 손실 함수로 훈련된 우리의 방법은 픽셀당 손실함수로 훈련된 방법과 비교하여 질 높은(fine) 세부사항을 더 잘 재구성(reconstruct)할 수 있다.

# 2. Related Work

# 2.1. Feed-forward image transformation
- 최근에, 다양한 feedforward 이미지 변환 작업은 해결되었다. 픽셀당 손실함수로 깊은 컨볼루젼 신경망을 훈련시킴으로써
- 의미론적 분할 방법(semantic segmentation method)은 입력 이미지에 대한 완전한 컨볼루젼 방식(fully convolutional manner)으로 네트워크를 작동(run)시키고, 픽셀당 분류 손실함수(per-pixel classification loss)로 훈련함으로써, 밀도 있는 장면 레이블(dense scene label)을 생성한다.
- 논문 "Zheng, S., Jayasumana, S., Romera-Paredes, B., Vineet, V., Su, Z., Du, D., Huang, C., Torr, P.H.: Conditional random fields as recurrent neural networks. In: Proceedings of the IEEE International Conference on Computer Vision. (2015) 1529–1537"은 픽셀당 손실함수(per-pixel loss)를 뛰어넘었다. 나머지 네트워크와 공동으로 훈련된 반복되는(recurrent) 레이어로써 CRF inference를 구성함으로써.
- 변환 네트워크(transformation network)의 구조(architecture)는 논문 "Long, J., Shelhamer, E., Darrell, T.: Fully convolutional networks for semantic segmentation. CVPR (2015)", "Noh, H., Hong, S., Han, B.: Learning deconvolution network for semantic segmentation. arXiv preprint arXiv:1505.04366 (2015)"에 영감을 얻었다. 네트워크상의(in-network) 다운샘플링(downsampling)을 사용하여 feature map의 공간적인 크기(spatial extent)를 줄였다. 그 feature map은 마지막 출력 이미지를 생성하는 네트워크상의 업샘플링(upsampling)에 의한 것이다.
- 깊이(depth), 표면 정규 추정(surface normal estimation)의 최신의 방법은 그들이 픽셀당 회귀(regression) 또는 분류 손실함수를 사용하여 훈련된 feed-forward 컨볼루젼 네트워크를 사용하여 컬러 입력 이미지를 기하학적인 의미있는 출력 이미지로 변환하는 것과 비슷하다.
- 몇몇의 방법은 픽셀당 손실함수를 넘어서 이동한다. 이미지 기울기(image gradient)에 패널티를 부여하거나(penalize) CRF 손실 레이어를 사용하여 출력 이미지의 local 일관성(consistency)를 강화함(enforce)으로써.
- feed-forward 모델은 픽셀당 손실함수를 사용하여 훈련되었다. 회색조(grayscale) 이미지를 컬러 이미지로 변환하기 위해.


# 2.2. Perceptual optimization(지각 최적화)
- 최근의 여러 논문은 최적화(optimization)를 사용하여 이미지를 생성했다. 객체는 지각적이고, 컨볼루젼 네트워크에서 추출된 고수준의 feature에 의존한다.
- 이미지는 훈련된 네트워크에서 인코딩된 기능을 이해하기 위해서 클래스 예측 점수(class prediction score) 또는 개별 특징(individual feature)을 최대화하여 생성될 수 있다.
- 유사한 최적화 기술은 신뢰도(confidence)가 높은 가짜(fooling) 이미지 또한 생성할 수 있다.
- Mahendran, Vedaldi는 다른 네트워크 레이어를 포함(retain)하는 이미지 정보를 이해하기 위해서 특징 재구성 손실(feature reconstruction loss)을 최소화함으로써 컨볼루젼 네트워크에서부터 feature를 반전(invert)시켰다. 유사한 방법은 이전에 local binary descriptor, HOG feature를 반전시키는데 사용됬었다.
- Dosovitskiy, Brox의 작업은 특히 우리의 작업과 관련이 있다. 그들은 feed-forward 신경망을 convolutional feature를 반전시키기 위해 훈련시켰기 때문이다. 최적화 문제의 해결책을 신속하게 근사시킴으로써.
- 그러나, 그들의 feed-forward 네트워크는 픽셀당 재구성 손실함수로 훈련된 반면에, 우리의 네트워크는 feature 재구성 손실 함수를 직접적으로 최적화한다.

# 2.3. Style Transfer(스타일 전송)
- Gatys 등등은 한 이미지의 content를 다른 이미지의 style과 결합함으로써 예술적 스타일 전송(artistic style transfer)를 실행했다. 특징 재구성 손실함수(feature reconstruction loss)와 스타일 재구성 손실함수(style reconstruction loss)을 공동으로 최소화함으로써. 여기서 스타일 재구성 손실함수는 사전학습된 컨볼루젼 네트워크로부터 추출된 feature에 기반한다. ; 유사한 방법이 이전의 텍스쳐 합성(texture synthesis)에 사용되었다.
- 이들의 방법은 고품질의 결과를 생성한다. 하지만, 그들의 방법은 계산적으로 비싸다(computationally expensive). 왜냐하면, 그 최적화 문제의 각 단계는 사전훈련된 네트워크를 통해서 전진(forward), 후진(backward) 통과가 필요하기 때문이다.
- 이러한 계산적인 부담(computational burden)을 극복하기 위해서, 우리는 feed-forward 네트워크를 훈련시킨다. 그들의 최적화 문제를 위한 해결책에 거의 근사치를 내기 위해서.

# 2.4. Image super-resolution(이미지 초고해상도)
- 이미지 초고해상도는 다양한 기술이 개발된 고전적인 문제(classic problem)이다.
- Yang 등등은 컨볼루젼 신경망이 널리 채택되기 전에 널리 사용되는 기술에 대한 철저한 평가(exhaustive evaluation)를 제공한다.
- 그들은 초해상도 기술(super-resolution technique)을 예측기반 방법(prediction-based method), 엣지기반 방법(edgebased method), 통계적 방법(statistical method), 패치기반 방법(patch-based method), 희소사전 방법(sparse dictionary method)으로 무리지었다. (bilinear, bicubic, Lanczos 알고리즘)
- 최근에, 픽셀당 유클리디안 손실함수(per pixel Euclidean loss)로 훈련된 3-layer 컨볼루젼 신경망을 사용하여 단일 이미지 초고해상도에서 우수한 성능을 달성했다.

# 3. Method
- 그림 2에서 보이듯이, 우리의 시스템은 2개의 요소(component)와 일치한다. : 이미지 변환 네트워크(image transformation network) $f_W$, 손실 네트워크(loss network) $\phi$. 여러 손실 함수 $l_1, ..., l_k$를 정의하는 데 사용된다.
- <img src="https://blog.kakaocdn.net/dn/cP3KFR/btqEcGCMxki/KkduOGoEPBJDkkGHuEpPq1/img.png" width=800>

  - 그림 2.
  - 시스템 개요(system overview)
  - 우리는 입력 이미지를 출력 이미지로 변환하기 위해서 이미지 변환 네트워크를 훈련시킨다.
  - 이미지 분류를 위해 사전학습시킨 loss network를 사용하여 지각 손실 함수(perceptual loss function)를 정의한다. 그 지각 손실 함수는 이미지간의 content, style의 지각적인 차이를 측정한다.
  - loss network는 훈련 과정동안 고정된 상태로 유지된다.
- 이미지 변환 네트워크(image transformation network)는 가중치 W에 의해 매개변수화된 깊은 잔여 컨볼루젼 신경망(deep residual convolutional neural network)이다.; 이미지 변환 네트워크는 입력 이미지 $x$를 출력 이미지 $\hat{y}$로 변환시킨다. mapping $\hat{y} = f_W(x)$를 통해서.
- 각각의 손실 함수는 $l_{i}(\hat{y}, y_{i})$ 스칼라값을 계산한다. 출력 이미지 $\hat{y}$과 타겟 이미지 $y_{i}$ 간의 차이를 측정함으로써.
- 이미지 변환 네트워크는 손실 함수의 가중치 조합을 최소화하기 위해 확률적 경사 하강법을 사용하여 훈련된다. : $$\large{W^{∗} = arg min_{W}\mathbf{E}_{x, \{y_{i}\}} \left[ \sum_{i=1}\lambda_{i}l_{i}(f_{W}(x), y_{i}) \right]}\qquad(1) $$
- 픽셀당 손실함수의 단점을 해결하고 이미지들간의 지각적, 의미론적인 차이를 우리의 손실 함수가 더 잘 측정할 수 있도록 하기 위해서, 우리는 최적화를 통해 이미지를 생성하는 최근의 작업에서 영감을 얻는다.
- 이러한 방법에서의 핵심 통찰력(key insight)은 이미지 분류를 위해 사전 훈련된 컨볼루젼 신경망이 이미 학습되었다는 것이다. 손실 함수에서 측정하려는 지각적, 의미론적인 정보를 인코딩하기 위해서.
- 그러므로, 우리는 손실 함수를 정의하기 위해서, 고정된 손실 네트워크로써, 이미지 분류를 위한 사전 훈련된 네트워크 $\phi$를 사용한다.
- 우리의 심층 컨볼루젼 변환 네트워크(deep convolutional transformation network)는 심층 컨볼루젼 네트워크인 손실 함수를 사용하여 훈련시킨다.
- 이미지간의 content, style 사이의 차이를 측정하는 특징 재구성 손실(feature reconstruction loss) $l_{feat}^{\phi}$과 스타일 재구성 손실(style reconstruction loss) $l_{style}^{\phi}$을 정의하는 데 손실 네트워크(loss network) $\phi$가 사용된다.
- 각각의 입력 이미지 $x$에 대해서, 우리는 content target $y_c$와 style target $y_s$가 있다.
- style transfer의 경우, content target $y_c$는 입력 이미지 $x$이고, 
- 출력 이미지 $\hat{y}$는 $x = y_c$의 content와 $y_s$의 style을 를 결합해야만 한다.; 우리는 style target 하나당 1개의 네트워크를 훈련시킨다.
- 단일 이미지 초고해상도의 경우, 입력 이미지 $x$는 저해상도 입력이다. content target $y_c$는 지상 실측 고해상도 이미지이다. 스타일 재구성 손실함수는 사용되지 않는다. ; 우리는 초고해상도 계수(factor) 하나당 1개의 네트워크를 훈련시킨다.


# 3.1. Image Transformation Networks(이미지 변환 네트워크)
- 우리의 이미지 변환 네트워크는 대략 Radford 등등에 의한 구조 기준(architectural guideline)을 따른다.
- 우리는 어떠한 풀링 레이어를 사용하지 X. 대신, 네트워크상의 다운샘플링, 업샘플링을 위해 스트라이드, 아주 약간의 스트라이드된 컨볼루젼을 사용한다.
- 우리의 네트워크 본체는 논문 "Gross, S., Wilber, M.: Training and investigating residual nets. http://torch.ch/blog/2016/02/04/resnets.html (2016)"의 구조를 사용하는 5개의 residual block으로 구성된다.
- 모든 non-residual convolutional layer에는 공간적 배치 정규화(spatial batch normalization)와 ReLU 비선형성(nonlinearity)이 따라온다. 출력 레이어를 제외하고 스케일링된 tanh 함수를 사용하여 출력 이미지가 [0, 255] 범위의 픽셀값을 갖도록 만든다.
- 9x9 커널을 사용하는 1번째, 마지막 레이어를 제외하고, 모든 컨볼루젼 레이어는 3x3 커널을 사용한다.
- 모든 우리의 네트워크의 정확한 구조는 보충 자료에서 찾을 수 있다.

# 3.1.1. Inputs and Outputs
- 스타일 전송의 경우, 입력 및 출력은 모두 3x256x256 shape의 컬러 이미지이다.
- 업샘플링 계수(factor)가 $f$인 초고해상도의 경우, 출력은 3x288x288 shape의 고해상도 이미지 패치이다. 입력은 3x288/$f$x288/$f$ shape의 저해상도 패치이다.
- 이미지 변환 네트워크는 fully convolutional이기 때문에, 테스트할 때, 이미지 변환 네트워크는 모든 해상도의 이미지에 적용할 수 있다.

# 3.1.2. Downsampling and Upsampling
- 업샘플링 계수(upsampling factor)가 $f$인 초해상도의 경우, 우리는 여러개의 잔차 블록(residual block)을 사용했다. stride 1/2의 $log_{2}f$ 컨볼루젼 레이어에 의한.
- 이것은 네트워크로 전달하기 전에 저해상도(low-resolution) 입력을 업샘플링하기 위해 bicubic 보간법(interpolation)을 사용하는 것과는 다르다.
- 고정된 업샘플링 함수에 의존하는 것 대신에, 아주 약간 스트라이드된 컨볼루젼은 나머지 네트워크와 함께 공동으로 학습할 수 있는 업샘플링 함수를 허락한다.
- 스타일 전송을 위해, 우리의 네트워크는 2개의 stride-2 컨볼루젼을 사용한다. 몇 개의 잔여 블록(residual block)에 의한 입력을 다운샘플링하기 위해서. 그리고, stride 1/2인 2개의 컨볼루젼 레이어를 업샘플링하기 위해서.
- 입력과 출력의 크기는 같지만, 다운샘플링을 한 다음에 업샘플링을 하는 네트워크에는 몇 가지 이점들이 있다.
- 이점 1. 계산(computational)
  - 순진한 구현(naive implementation)에서, $C × H × W$ 크기의 입력에서 $C$ 필터를 사용하는 3x3 컨볼루젼에는 $9HWC^2$ multiply-add가 필요하다. 이는 #$DC × H/D × W/D$ 크기의 입력에서 $DC$ 필터를 사용하는 3x3 컨볼루젼과 동일한 비용(cost)이다.
  - 따라서 다운샘플링 후에, 우리는 동일한 계산 비용(computational cost)으로 더 큰 네트워크를 사용할 수 있다.
- 이점 2. 효과적인 수용 필터의 크기(effective receptive field size)
  - 고품질 스타일 전송은 이미지의 많은 부분을 일관된(coherent) 방식으로 변경할 필요가 있다.; 그러므로 출력의 각 픽셀이 입력에 큰 효과적인 수용 필드를 갖는 것이 유리하다.
  - 다운샘플링이 없을 경우, 각각의 추가적인 3x3 컨볼루젼 레이어는 효과적인 수용 필드의 크기를 2씩 증가시킨다.
  - 계수 $D$로 다운샘플링이 일어난 후에, 각각의 3x3 컨볼루젼은 효과적인 수용 필드 크기를 2D만큼 증가시킨다. 동일한 수의 레이어로 더 큰 효과적인 수용 필드를 제공함으로써.

# 3.2. Perceptual Loss Functions(지각 손실함수)
- 우리는 2개의 지각 손실 함수(perceptual loss function)을 정의했다. 이미지간의 고수준의 지각과 의미론적인 차이를 측정하는.
- 그 2개의 지각 손실 함수는 이미지 분류(image classification)을 위해 사전학습된 loss network $\phi$를 사용한다. 즉, 이 지각 손실 함수들이 심층 컨볼루젼 신경망이(deep convolutional neural network)라는 의미이다.
- 우리의 모든 실험에서, $\phi$는 ImageNet 데이터셋에 대해 사전학습된 16-layer VGG network이다.

# 3.2.1. Feature Reconstruction Loss(특징 재구성 손실함수)
- 출력 이미지의 픽셀 $\hat{y} = f_{W}(x)$가 타겟 이미지의 픽셀 $y$와 정확하게 일치하는 것을 권장(encourage)하는 것 보다는, 우리는 대신 지각 손실 함수들이 손실 네트워크 $\phi$에 의해 계산된 것과 비슷한 특징 표현(feature representation)을 갖도록 권장한다.
- 이미지 $x$를 처리할 때 $\phi_{j}(x)$를 네트워크 $\phi$의 $j$번째 레이어의 활성화층(activation)이라고 가정하자.; 만약 $j$가 컨볼루젼 레이어라면, $\phi_{j}(x)$는 $C_{j} × H_{j} × W_{j}$ shape의 feature map이 된다.
- 특징 재구성 손실함수(feature reconstruction loss)는 특징 표현(feature representation)간의 (제곱(squared), 정규화(normalized)) 유클리디안 거리이다.: $$\large{l_{feat}^{\phi, j}(\hat{y}, y) = \frac{1}{C_{j}H_{j}W_{j}}\lVert \phi_{j}(\hat{y}) - \phi_{j}(y) \rVert_{2}^{2}}\qquad(2)$$
- 그림 3에서 보여준 것처럼, 초기 레이어(early layer)의 특징 재구성 손실함수를 최소화하는 이미지 $y$를 찾는 것은 $y$로부터 시각적으로 구별할 수 없는 이미지를 생성하는 경향이 있다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig3_HTML.gif" width=600>

  - 그림 3.
  - 우리는 이미지 $\hat{y}$를 찾기 위해 최적화를 사용한다. 사전학습된 VGG-16 손실 네트워크 $\phi$에서 여러 레이어 $j$에 대한 특징 재구성 손실함수 $l_{feat}^{\phi, j}(\hat{y}, y)$를 최소화하는.
  - 우리가 상위 레이어(higher layer)에서 재구성할 때, 이미지 content와 종합적인 공간 구조(spatial structure)는 보존된다. 하지만, 색상, 텍스쳐, 정확한 모양은 보존되지 않는다.
- 이미지 변환 네트워크를 훈련하기 위해서 특징 재구성 손실함수를 사용하는 것은 출력 이미지 $\hat{y}$가 타겟 이미지 $y$와 지각적으로 비슷하도록 권장한다. 하지만, 정확하게 일치하도록 강요하지는 않는다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig4_HTML.gif" width=600>

  - 그림 4.
  - 우리는 이미지 $\hat{y}$를 찾기 위해 최적화를 사용한다. 사전학습된 VGG-16 손실 네트워크 $\phi$에서 여러 레이어 $j$에 대한 특징 재구성 손실함수 $l_{feat}^{\phi, j}(\hat{y}, y)$를 최소화하는.
  - 이미지 $\hat{y}$는 스타일적 특징(stylistic feature)를 보존하지만 공간적인 구조(spatial structure)는 보존하지 않는다.

# 3.2.2. Style Reconstruction Loss(스타일 재구성 손실함수)
- 특징 재구성 손실함수는 출력 이미지 $\hat{y}$가 타겟 이미지 $y$의 content에서 벗어날 때 출력 이미지 $\hat{y}$에 불이익을 준다.
- 우리는 또한 스타일에서의 차이에 불이익을 주고자 한다.: 색상, 텍스쳐, 일반적인 패턴 등등
- 이러한 효과를 달성하기 위해서, Gatys 등등은 다음과 같은 스타일 재구성 손실함수를 제안했다.
- 위와 같이, $\phi_{j}(x)$가 입력이 $x$일 때 네트워크 $\phi$의 $j$번째 레이어의 활성화함수이다. 이것은 $C_{j} × H_{j} × W_{j}$ 모양의 feature map이다.
- Gram matrix $G_{j}^{\phi}(x)$를 다음과 같은 요소들로 주어진 $C_{j} × C_{j}$ 행렬로 정의한다.: $$\large{G_{j}^{\phi}(x)_{c, c^{\prime}} = \frac{1}{C_{j}H_{j}W_{j}}\sum_{h=1}^{H_{j}}\sum_{w=1}^{W_{j}}\phi_{j}(x)_{h, w, c}\phi_{j}(x)_{h, w, c^{\prime}}}\qquad(3)$$
- 만약 우리가 $\phi_{j}(x)$를 $H_{j} × W_{j}$ grid의 각 점에 대해 $C_{j}$-차원의 특징을 제공하는 것으로 해석한다면, $G_{j}^{\phi}(x)$는 8개의 Johnson 등등의 차원적인 특징(dimensional feature)들의 중심이 아닌 공분산(uncentered covariance)에 비례하게 된다. 각 grid 위치를 독립적인 샘플로 취급하면서.
- 따라서 그것은 함께 활성화되는 경향이 있는 특징(feature)에 대한 정보를 캡쳐한다.
- Gram matrix는 효과적으로 계산될 수 있다. $\phi_{j}(x)$을 $C_{j} × H_{j}W_{j}$ 모양(shape)의 행렬 $ψ$로 재구성함으로써.; 그러면 $$\large{G_{j}^{\phi}(x) = \frac{ψψ^{T}}{C_{j}H_{j}W_{j}}}$$가 된다.
- 그리고 스타일 재구성 손실함수는 출력 이미지와 타겟 이미지의 Gram matrix 사이의 차이의 squared Frobenius norm이 된다.: $$l_{style}^{\phi, j}(\hat{y}, y) = \lVert G_{j}^{\phi}(\hat{y} - G_{j}^{\phi}(y))\rVert_{F}^{2}\qquad(4)$$
- 스타일 재구성 손실함수는 $\hat{y}, y$이 다른 크기를 가지고 있을 경우에도 잘 정의되어있다. 왜냐하면, 그들의 Gram matrix들이 같은 모양(shape)를 갖기 때문이다.
- 그림 5에서 보여준 것처럼, 스타일 재구성 손실함수를 최소화하는 이미지 $\hat{y}$를 생성하는 것은 타겟 이미지의 스타일적 특징(stylistic feature)을 보존한다. 하지만, 타겟 이미지의 공간 구조(spatial structure)는 보존하지 못한다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig5_HTML.gif" width=700>

  - 그림 5.
  - 우리의 스타일 전송 네트워크와 Gatys 등등의 방법(논문 [10])은 동일한 목표(objective)를 최소화시킨다.
  - 50개의 이미지에서 그들의 목표값을 비교한다.; 파선과 오차 막대는 표준편차를 보여준다.
  - 우리의 네트워크는 256x256 이미지로 훈련되었지만, 더 큰 이미지로 일반화된다.
- 상위 레이어(higher layer)에서 재구성하는 것은 타겟 이미지의 더 큰 규모의 구조(larger-scale structure)가 전송된다.
- 단일 레이어 $j$가 아닌 레이어 세트 $J$에서 스타일 재구성을 수행하기 위해서, 우리는 $l_{style}^{\phi, J}(\hat{y}, y)$를 각각의 레이어 $j ∈ J$의 손실의 합으로 정의한다.

# 3.3. Simple Loss Functions(간단한 손실 함수들)
- 위에서 정의된 지각 손실함수(perceptual loss) 뿐만 아니라, 우리는 또한 낮은 수준의 픽셀 정보(low-level pixel information)에만 의존하는 2개의 간단한 손실함수를 정의한다.

# 3.3.1. Pixel Loss(픽셀 손실함수)
- 픽셀 손실함수는 출력 이미지 $\hat{y}$와 타겟 이미지 $y$ 사이의 (정규화된) 유클리디안 거리이다.
- 만약에 둘다 $C × H × W$ 모양을 가진다면, 픽셀 손실은 $$\large{l_{pixel}(\hat{y}, y) = \frac{\lVert \hat{y} - y\rVert_{2}^{2}}{CHW}}$$로 정의된다.
- 이것은 네트워크가 일치할 것으로 예상되는 지상실측 목표(ground-truth target)가 있을 때에만, 사용할 수 있다.

# 3.3.2. Total Variation Regularization(총 변동 정규화)
- 출력 이미지 $\hat{y}$를 장려하기 위해서, 우리는 특징 반전, 초고해상도에 대한 사전 작업을 따른다. 그리고, 총 변동 정규화 $l_{TV}(\hat{y})$를 사용한다.

# 4. Experiments(실험)
- 우리는 2개의 이미지 변환 작업(image transformation task)에 대한 실험을 한다.: 스타일 전송(style transfer), 단일 이미지 초고해상도(single image super-resolution)
- 스타일 전송에 대한 이전 작업은 이미지를 생성하기 위해 최적화를 사용했었다.; 우리의 feed-forward 네트워크는 비슷한 질적인 결과를 준다. 하지만, 우리의 feed-forward 네트워크는 3배 더 빠르다.
- 단일 이미지 초해상도에 대한 컨볼루젼 신경망을 사용한 이전 작업은 픽셀당 손실함수를 사용했다.; 우리는 지각 손실을 대신 사용함으로써 질적인 결과를 향상시켰다.

# 4.1. Style Transfer(스타일 전송)
- 스타일 전송의 목표는 target content image $y_{c}$의 content와 target style image $y_{s}$의 style을 결합한 이미지 $\hat{y}$를 생성하는 것이다.
- 우리는 몇개의 엄선된(hand-picked) 스타일 타겟에 대해 스타일 타겟 하나당 1개의 이미지 변환 네트워크를 훈련한다. 그리고, Gatys 등등의 기준 접근방식(baseline approach)과 비교한다.

# 4.1.1. Baseline(기준)
- 기준으로써, 우리는 Gatys 등등의 방법을 재구현(reimplement)한다.
- style target $y_{s}$, content target $y_{s}$, 특징(feature)과 스타일 재구성(style reconstruction)을 수행하기 위한 레이어 $i, j$가 주어진다면, 이미지 $\hat{y}$는 $$\large{
\hat{y} = argmin_{y} λ_{c} l_{feat}^{\phi, j} (y, y_{c}) + λ_{s} l_{style}^{\phi, J}(y, y_{s}) + λ_{TV} l_{TV}}\qquad(5) \\ λ_{c}, λ_{s}, λ_{T} : 스칼라 \\ y : 백색 잡음으로 초기화 \\ 최적화 : L-BFGS를 사용$$ 문제를 해결함으로써 생성된다.
- 우리는 방정식 5의 제한되지 않은 최적화는일반적으로 픽셀이 [0, 255]의 범위를 벗어나는 이미지를 생성한다는 것을 발견했다.
- 출력이 [0, 255] 범위로 제한되는 우리의 방법과의 좀 더 공정한 비교를 위해서, 기준(baseline)의 경우, 우리는 방정식 5를 최소화시킨다. 투영된(projected) L-VFGS를 사용함으로써. 각각의 반복(iteration)에서 [0, 255] 범위에 이미지 $y$를 고정(clip)함으로써
- 대부분의 경우에서, 최적화는 500번의 반복(iteration) 내에서 만족스러운 결과로 수렴된다.
- 이 방법은 느리다. 왜냐하면, 각각의 L-BFGS 반복은VGG-16 손실 네트워크 $\phi$를 통한 순방향과 역방향 통과가 필요하기 때문이다.

# 4.1.2. Training Details
- 우리의 스타일 전송은 Microsoft COCO 데이터셋으로 훈련되었다.
- 우리는 각각의 8만개의 훈련 이미지를 256x256 크기로 재조정했다. 그리고 40000번의 반복에 대해 배치 사이즈를 4로 하여 우리의 네트워크에 훈련시켰다. 대략 훈련 데이터에 대해 2 epoch만큼을 주면서.
- 우리는 학습률이 $1×10^{-3}$인 Adam 최적화 방법을 사용한다.
- 출력이미지는 총 변동 정규화에 의해 정규화되었다. 스타일 타겟별 교차 검증(cross-validation)을 통해 선택된 $1×10^{-6}$과 $1×10^{-4}$ 사이의 강도로.
- 우리는 가중치 감소(weight decay)나 드롭아웃(dropout)을 사용하지 않는다. 왜냐하면 모델은 2 epoch 내에 과적합되지 않기 때문이다.
- 모든 스타일 전송 실험에서, 우리는 relu2_2 레이어의 특징 재구성 손실함수와 relu1_2, relu2_2, relu3_3, relu4_3 레이어의 스타일 재구성 손실함수를 계산했다. VGG-16 손실 네트워크 $\phi$의.
- 우리의 구현(implementation)은 Roch, cuDNN을 사용한다. ; 교육은 1개의 GTX Titan X GPU에서 대략 4시간정도 걸린다.

# 4.1.3. Qualitative Results(질적인 결과)
- 그림 6에서, 우리는 다양한 스타일과 콘텐츠 이미지에 대한 기준 방법(baseline method)의 결과와 우리의 결과를 비교함으로써 질적인 예시를 보여준다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig6_HTML.gif" width=700>

  - 그림 6.
  - 우리의 이미지 변환 네트워크를 사용하여 스타일 전송 결과의 예시
  - 우리의 결과는 Gatys 등등의 방법과 질적으로 비슷하다. 하지만, 생성하는데 걸리는 속도가 훨씬 빠르다. (표 1)
  - 생성된 모든 이미지는 256x256 픽셀이다.

- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fok1nR%2FbtqEcGQkJQv%2FWleQI5U98FMQRKH1Zw8Po1%2Fimg.png" width=700>

  - 표 1.
  - 우리의 스타일 전송 네트워크의 속도(초) VS 다양한 반복횟수(iteration)과 이미지 해상도(resolution)에 대한 최적화를 기반으로 한(optimization-based) 기준(baseline)
  - 우리의 방법은 비슷한 질적인 결과를 준다.(그림 6) 하지만, 기준 방법의 단일 최적화 단계보다 더 빠르다.
  - 두 방법 모두 GTX Titan X GPU에서 벤치마킹(benchmark)되었다.

- 모든 경우에서, 하이퍼파라미터 $λ_c, λ_s, λ_{TV}$는 2가지 방법에서 정확하게 일치한다.; 모든 콘텐츠 이미지는 MS-COCO 2014 검증 데이터셋으로부터 가져온다.
- 전반적으로, 우리의 결과는 기준 방법과 질적으로 비슷하다.
- 우리의 방법은 256x256 이미지로부터 학습되었음에도 불구하고 테스트 시간에 모든 크기의 이미지에도 fully-convolutional 방법을 적용할 수 있다.
- 그림 7에서, 우리는 우리의 모델을 512x512 이미지에 사용한 스타일 전송의 예시를 보여준다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig7_HTML.jpg" width=700>

  - 그림 7.
  - 512x512 이미지에 스타일 전송 결과 예시.
  - 이 모델은 테스트 시간에 고해상도 이미지에 fully convolutional 방식으로 적용된다.
  - 스타일 이미지는 그림 6과 동일하다.

- 이 결과에서 훈련된 스타일 전송 네트워크가 이미지의 의미론적인 내용을 인식하고 있음이 분명하다.
- 예를 들어 그림 7의 해변 이미지에서, 사람들은 변환된 이미지에서 명확하게 인식된다. 하지만, 배경은 인식할 수 없을 정도로 뒤틀려있다.; 고양이 이미지에서도 비슷하게, 변형된 이미지에서는 고양이의 얼굴은 선명하다. 하지만, 몸은 그렇지 않다.
- 한가지 설명은 VGG-16 손실 네트워크가 사람과 동물에 대해 선택적인 특징을 갖고 있다고 한다. 왜냐하면, 이러한 객체들은 학습된 분류 데이터셋에 있기 때문이다.
- 우리의 스타일 전송 네트워크는 VGG-16 특징을 보존하도록 훈련되어있다. 그리고 그렇게 함으로써, 그들은 배경 물체들보다 사람과 동물들을 보존하는 방법에 대해 학습한다.

# 4.1.4. Quantitative Results(양적인 결과)
- 기준 방법과 우리의 방법은 모두 방정식 5를 최소화한다.
- 기준 방법은 출력 이미지에 대해 명백한(explicit) 최적화를 수행한다. 하지만, 우리의 방법은 단일 순방향 흐름에서 콘텐츠 이미지 $y_c$에 대한 해결방법을 찾도록 훈련되었다.
- 그러므로 우리는 아마도 2가지 방법을 양적으로(quantitatively) 비교할 수 있다. 방정식 5를 성공적으로 최소화하는 정도를 측정함으로써.
- 우리는 MS-COCO 검증 데이터셋으로부터 50개의 이미지에 대해 우리의 방법과 기준 방법을 실행한다. Pablo Picasso의 The Muse를 스타일 이미지로 사용하면서.
- 기준 방법에서, 우리는 최적화의 각각의 반복(iteration)에서 목적 함수의 값을 기록한다. 그리고, 우리의 방법에서, 우리는 각각의 이미지에 대해 방정식 5의 값을 기록한다.; 우리는 또한 $y$가 콘텐츠 이미지 $y_c$와 동일할 때, 방정식 5의 값도 계산한다.
- 결과는 그림 5에 있다.
- 우리는 콘텐츠 이미지 $y_c$가 매우 높은 손실을 달성하고, 우리의 방법이 명백한 최적화의 50~100회 반복에 맞먹는 손실을 달성한다는 것을 보여준다.
- 우리의 네트워크는 256x256 이미지에 대해 방정식 5를 최소화하도록 훈련되었지만, 더 큰 이미지에 적용될 때, 목표를 최소화하는 데에도 성공했다.
- 우리는 512x512, 1024x1024에서 50개의 이미지에 대한 동일한 양적인 평가를 반복한다.; 결과는 그림 5에 나와있다.
- 더 높은 해상도에서, 우리의 모델이 기준 방법의 50~100회 반복에 맞먹는 손실을 달성한다.

# 4.1.5. Speed(속도)
- 표 1에서, 우리는 다양한 이미지 크기에 대해 우리의 방법과 기준 방법의 실행 시간(runtime)을 비교한다.; 우리는 기준방법에서의, 다양한 최적화 반복 횟수에 대한 시간을 보고한다(report).
- 모든 이미지 크기에서, 우리의 방법의 실행시간은 기준 방법의 단일 반복의 속도의 대략 2배이다.
- 기준 방법의 500회 반복과 비교하여, 우리의 방법은 3배나 더 빠르다.
- 우리의 방법은 20FPS에서 512x512크기의 이미지를 처리한다. 실시간 또는 비디오에서 스타일 전송하는 것이 실현가능하도록 만듦으로써.

# 4.2. Single-Image Super-Resolution(단일 이미지 초고해상도)
- 단일 이미지 초고해상도에서, 작업은 저해상도 입력에서 고해상도 출력 이미지를 생성하는 것이다.
- 이것은 본질적으로 부적절한 문제(ill posed problem)이다. 왜냐하면, 각각의 저해상도 이미지에 대해서, 그것을 생성할 수 있는 여러 개의 고해상도 이미지가 있기 때문이다.
- 초고해상도 요인이 증가함에 따라 모호성(ambiguity)은 더욱 심해진다.; 큰 요인(x4, x8)의 경우, 고해상도 이미지의 질 높은 세부사항은 저해상도 버전에서 근거가 거의 또는 전혀 없을 수도 있다.
- 이 문제를 해결하기 위해서, 우리는 고해상도 네트워크를 훈련시킨다. 심층 신경망(deep convolutional neural network)을 사용한 이미지 초고해상도를 사용하는 픽셀당 손실을 사용하지 않고, 특징 재구성 손실을 사용한다. (Section 3) 사전훈련된 손실 네트워크에서 초고해상도 네트워크로 의미론적인 지식을 전송하기 위해서.
- 우리는 x4, x8 초고해상도에 집중한다. 왜냐하면 인자(factor)가 많을수록 입력에 대한 의미론적 추론(semantic reasoning)이 더 필요하기 때문이다.
- 초고해상도를 평가하는 데 사용되는 전통적인 척도(metric)는 PSNR, SSIM이다. 둘 다 인간의 시각 품질 평가와 연관성이 없는 것으로 밝혀졌다.

- PSNR, SSIM은 픽셀간 낮은 수준 차이에만 의존하고, 초고해상도에는 유효하지 않을 수 있는 덧셈의(additive) 가우시안 노이즈(Gaussian noise)를 가정하여 작동한다.
- 또한, PSNR은 픽셀당 손실 $l_{pixel}$과 동일하다. 그러므로, PSNR로 측정할 때 픽셀당 손실을 최소화하도록 훈련된 모델은 특징 재구성 손실을 최소화하도록 훈련된 모델보다 성능이 항상 뛰어나야한다.
- 그러므로 우리는 이러한 실험의 목표가 최신의 PSNR, SSIM 결과를 달성하는 것이 아닌, 픽셀당 손실과 특징 재구성 손실로 훈련된 모델 간의 질적인 차이를 보여주는 것이라고 강조한다.


# 4.2.1. Model Details
- VGG-16 손실 네트워크 $\phi$의 레이어 relu2_2에서 특징 재구성 손실을 최소화함으로써 x4, x8 초고해상도를 수행하도록 모델을 훈련시킨다.
- MS-COCO 훈련 데이터셋의 1만개의 이미지로부터 288x288 패치로 훈련한다. 그리고, 폭 σ=1.0의 가우시안 커널로 흐릿하게 만들고 bicubic 보간법으로 다운샘플링함으로써 저해상도 입력을 준비한다.
- 가중치 감소(weight decay)나 드롭아웃(dropout)없이 $1×10^{-3}$의 학습률로 Adam 최적화 방법을 사용하여 20만회 반복에 대해 배치 크기 4로 훈련시킨다.
- 사후 처리 단계로써, 네트워크 출력과 저해상도 입력 간의 히스토그램 매칭을 수행한다.

# 4.2.2. Baselines(기준)
- 기준 모델로써, 우리는 최신의 성능을 내는 SRCNN을 사용한다.
- SRCNN은 ILSVRC 2013 detection 데이터셋의 33x33 패치에서 픽셀당 손실을 최소화하도록 훈련된 3-layer 컨볼루젼 네트워크이다.
- SRCNN은 x8 초고해상도에 대해 훈련되지 않았으므로 x4에서만 평가할 수 있다.
- SRCNN은 109회 이상의 반복에 대해 훈련되었다. 이는 우리 모델에서 계산적으로 실현가능하지 않다.
- 데이터, 학습, 구조에서 SRCNN과 우리 모델 간의 차이점을 설명(account)하기 위해서, 우리는 $l_{pixel}$을 사용하여 x4, x8 초고해상도에 대한 이미지 변환 네트워크를 훈련시킨다.; 이러한 네트워크는 $l_{feat}$을 최소화하도록 훈련된 네트워크와 동일한 데이터, 구조, 훈련을 사용한다.

# 4.2.3. Evaluation(평가)
- 우리는 표준 Set5, Set14, BSD100 데이터셋의 모든 모델을 평가한다.
- 우리는 PSNR, SSIM을 보고한다. YCbCr 색 공간(colorspace)으로 변환한다음 Y 채널에 대해서만 PSNR, SSIM을 계산한다.

# 4.2.4. Results
- 우리는 그림 8에서 x4 초고해상도에 대한 결과를 보여준다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig8_HTML.gif" width=500>

  - 그림 8.
  - Set5(위 그림), Set14(아래 그림)의 이미지에서 x4 초고해상도에 대한 결과이다.
  - 각 예제에 대한 PSNR/SSIM 값과 각 데이터셋의 평균을 보고한다.
  - 더 많은 결과는 보충 자료에 나와있다.
- 다른 방법과 비교할 때, 특징 재구성을 위해 훈련된 우리의 모델은 1번째 이미지의 속눈썹과 2번째 이미지의 모자의 개별 요소와 같은 날카로운 엣지와 질 높은 세부사항을 재구성할 때 매우 효과적이다.
- 특징 재구성 손실함수는 확대율(배율, magnification)에서 볼 수 있는 약간의 교차해치(cross-hatch) 패턴을 발생시킨다. 기준 방법과 비교하여 PSNR, SSIM을 손상시킨다.
- x8 초고해상도의 결과는 그림 9에 나와있다.
- <img src="https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-46475-6_43/MediaObjects/419974_1_En_43_Fig9_HTML.gif" width=700>

  - 그림 9.
  - BSD100 데이터셋 이미지에서 배율 계수(scale factor) x8을 사용한 초고해상도의 결과.
  - 우리는 예제 이미지에 대한 PSNR/SSIM과 각 데이터셋에 대한 평균값을 보고한다.
  - 더 많은 결과는 보충 자료에 나와있다.
- 우리의 $l_{feat}$ 모델이 다른 모델과 비교하여 엣지와 질 높은 세부사항에서 잘 작동한다는 것을 알 수 있다. 말의 다리와 말발굽과 같은.
- $l_{feat}$ 모델은 가장자리를 마구잡이로 날카롭게 하지 않는다.; $l_{pixel}$ 모델과 비교하여, $l_{feat}$ 모델은 말과 말을 탄 사람의 경계 가장자리를 선명하게 해준다. 하지만, 배경의 나무는 산만한(diffuse) 상태로 유지된다. $l_{feat}$ 모델은 아마도 이미지 의미를 더 잘 인식할 수 있음을 시사한다.
- 우리의 $l_{pixel}$과 $l_{feat}$ 모델은 동일한 구조, 데이터, 훈련 과정을 공유하기 때문에, 두 모델간의 모든 차이점은 $l_{pixel}$과 $l_{feat}$ 손실의 차이 때문이다.
- $l_{pixel}$ 손실은 시각적인 인공물(artifact)가 적고 PSNR값이 높다. 하지만, $l_{feat}$ 손실은 질 높은 세부사항을 재구성하는데 더 효과적이고 시각적인 결과를 만족스럽게 만든다.

# 5. Conclusion
- 이 논문에서, 우리는 feed-forward 이미지 변환 작업의 이점과 이미지 생성의 최적화를 기반으로한 방법을 결합시켰다. 지각 손실 함수를 사용하여 feed-forward 변환 네트워크를 훈련시킴으로써.
- 우리는 비슷한 성능을 달성하고 존재했던 방식과 비교하여 엄청나게 속도를 향상시킨 스타일 전송과, 단일 이미지 초고해상도를 적용했다. 여기서 우리는 지각 손실을 이용한 훈련을 통해 모델이 더 질 높은 세부사항과 가장자리를 더 잘 재구성할 수 있음을 보여줬다.
- 향후 작업에서, 우리는 색상화와 의미론적 분할과 같은다른 이미지 변환 작업에 대한 지각 손실 함수의 사용을 연구하고자 한다.
- 우리는 또한 다양한 손실 네트워크의 사용을 조사할 예정이다. 예를 들어, 손실 네트워크가 다른 작업에 훈련되었든, 다른 데이터셋에 훈련되었든지 간에, 다양한 유형의 의미론적 지식을 사용하여 이미지 변환 네트워크에 전할 수 있는 지의 여부.

---
---
[CartoonGAN 논문 추가 개념 정리](https://github.com/yoonchaiyoung/ML-DL_Study/blob/master/%EB%AA%A8%EB%A5%B4%EB%8A%94%20%EA%B0%9C%EB%85%90%20%EA%B2%80%EC%83%89%ED%95%98%EC%97%AC%20%EC%A0%95%EB%A6%AC.ipynb)